Data Source: https://www.kaggle.com/competitions/fake-news


Title: input features

Label: output

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
news_df = pd.read_csv('/content/drive/Othercomputers/My Laptop/syr_ads_deep_learning/labs/datasets/fake_news_kaggle/train.csv.zip')
news_df.shape

(20800, 5)

In [ ]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
news_df = news_df.dropna()

In [ ]:
x_train = news_df['title']
y_train = news_df['label']

(x_train.shape, y_train.shape)

((18285,), (18285,))

# Creating the model

![A drawing of the information flow in the model](https://www.tensorflow.org/static/text/tutorials/images/bidirectional.png)

Onehot representation

In [ ]:
vocab_size = 5000
STOP_WORDS = stopwords.words('english')
stemmer = PorterStemmer()
corpus = []

for review in x_train:
  review = re.sub('[^a-zA-Z0-9]', ' ', review)
  review = review.lower()
  review = review.split()
  review = [stemmer.stem(word) for word in review if word not in STOP_WORDS]
  review = ' '.join(review)
  corpus.append(review)

assert len(corpus) == x_train.shape[0]

In [ ]:
one_hot_repr = [tf.keras.preprocessing.text.one_hot(review, vocab_size) for review in corpus]
assert len(one_hot_repr) == x_train.shape[0]

In [ ]:
one_hot_repr[:5]

[[3661, 4948, 232, 677, 3805, 3671, 779, 2556, 461, 4414],
 [3645, 342, 3689, 2991, 3968, 3687, 4625],
 [2939, 3956, 3160, 1006],
 [3138, 2105, 4476, 2256, 127, 3403, 4797],
 [4306, 3968, 751, 4095, 4005, 4425, 3968, 397, 4015, 4924]]

All the sentece length are different. Before embedding, we need to make this sentence of equal length.

Padding:

In [ ]:
sent_length = 20
padded_reviews = tf.keras.preprocessing.sequence.pad_sequences(one_hot_repr, padding='pre', maxlen=sent_length)
assert len(padded_reviews) == x_train.shape[0]

In [ ]:
padded_reviews

array([[   0,    0,    0, ..., 2556,  461, 4414],
       [   0,    0,    0, ..., 3968, 3687, 4625],
       [   0,    0,    0, ..., 3956, 3160, 1006],
       ...,
       [   0,    0,    0, ..., 3905, 4160,  650],
       [   0,    0,    0, ...,  285, 1835,  708],
       [   0,    0,    0, ..., 4543,  742, 1183]], dtype=int32)

Creating the model

In [ ]:
embedding_vector_features = 40
fake_news_clf = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_vector_features, input_length=sent_length),
    keras.layers.SimpleRNN(100),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid'),
])

fake_news_clf.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 100)               14100     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 214,201
Trainable params: 214,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_train

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [ ]:
fake_news_clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fake_news_clf.fit(padded_reviews, y_train, validation_split=0.33, epochs=10, batch_size=64, verbose=True)

Epoch 1/10
192/192 [==============================] - 6s 20ms/step - loss: 0.2986 - accuracy: 0.8636 - val_loss: 0.1945 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1054 - accuracy: 0.9617 - val_loss: 0.2277 - val_accuracy: 0.9031
Epoch 3/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0298 - accuracy: 0.9919 - val_loss: 0.2969 - val_accuracy: 0.9079
Epoch 4/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0081 - accuracy: 0.9984 - val_loss: 0.3514 - val_accuracy: 0.9100
Epoch 5/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0019 - accuracy: 0.9999 - val_loss: 0.3998 - val_accuracy: 0.9087
Epoch 6/10
192/192 [==============================] - 5s 23ms/step - loss: 5.6687e-04 - accuracy: 1.0000 - val_loss: 0.4294 - val_accuracy: 0.9064
Epoch 7/10
192/192 [==============================] - 3s 17ms/step - loss: 3.0468e-04 - accuracy: 1.0000 - val_loss: 0.4542 - val_accuracy

Training accuracy is 100% which could be due to overfitting